<a href="https://colab.research.google.com/github/blainerothrock/nlp-group-2/blob/master/nlp_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Assignment 2 (Bengio and other Neural Language Models)

In [1]:
from __future__ import absolute_import, division, print_function

# %tensorflow_version 2.x
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print(tf.__version__)

# from google.colab import drive, files 
# drive.mount('/content/drive')

import os, pickle
import numpy as np
import math
import typing

Instructions for updating:
non-resource variables are not supported in the long term
2.0.0


In [5]:
blaine_ubuntu_data_path = '../data'
blaine_data_path = '/Users/blaine/Google Drive File Stream/My Drive/Winter20/nlp/nlp_group2/data'
data_path = 'drive/My Drive/Winter20/nlp/nlp_group2/data'
grant_data_path = 'drive/My Drive/nlp_group2/data'
sundar_data_path = 'drive/My Drive/nlp_group2/data'
z_data_path = 'drive/My Drive/nlp_group2/data'
sundar_local_path = '~/Workspaces/Q2/NLP/data'

In [6]:
data_path = blaine_ubuntu_data_path

In [7]:
print(os.listdir(data_path))

['._brown_tokenized.txt', '._group2.test.txt', '._group2.tagged_valid.txt', 'group2.raw.txt', 'group2.int_test.p', 'brown_test_cost_history.p', 'group2.int_valid.p', 'group2.valid.txt', '._group2.test.p', 'group2.int_train.p', '._group2.int_tagged_valid.p', 'brown_val_perplexity_history.p', 'brown_perplexity_history.p', '._group2.tagged_train.p', 'group2.int_tagged_test.p', '._group2.train.txt', 'group2.tagged_vocab.p', 'group2.valid.p', 'group2.tagged_test.p', 'group2.tagged_test.txt', 'brown_cost_history.p', '._group2.int_valid.p', '._group2.valid.p', '._group2.int_tagged_train.p', '._group2.int_train.p', 'group2.tagged_train.txt', '._group2.tagged_train.txt', 'group2.test.txt', 'group2.test.p', 'group2.tagged_valid.txt', '._group2.vocab_dict.p', '._group2.tagged_test.txt', '._group2.tagged_vocab_dict.p', '._group2.tagged_vocab.p', '._Icon\uf00d', '._group2.valid.txt', 'group2.vocab.p', '._group2.tagged_valid.p', 'brown_tokenized.txt', 'group2.int_tagged_valid.p', '._group2.int_test.

## Task 1
Split train corpus with `batch_size=30` and `window=5` 

In [8]:
# load data
int_train = pickle.load(open(os.path.join(data_path, 'group2.int_train.p'), 'rb'))
int_validation = pickle.load(open(os.path.join(data_path, 'group2.int_valid.p'), 'rb'))
int_test = pickle.load(open(os.path.join(data_path, 'group2.int_test.p'), 'rb'))
train = pickle.load(open(os.path.join(data_path, 'group2.train.p'), 'rb'))
vocab_dict = pickle.load(open(os.path.join(data_path, 'group2.vocab_dict.p'), 'rb'))
# int_train = [vocab_dict[w] for w in train]

In [9]:
print(int_train[:10], '\n', train[:10])

print("vocab len: %i" % len(vocab_dict))
print("int rep: %s" % len(int_train))
print("train token: %s" % len(train))

[18050, 9033, 16614, 35216, 3311, 34744, 19712, 39720, 38681, 5003] 
 ['<s>', 'the', 'battle', 'of', 'fort', 'donelson', 'was', 'fought', 'from', 'february']
vocab len: 42314
int rep: 4628458
train token: 4628458


In [10]:
# batch the train integer representations
def gen_batches(context_size, num_batches, data):
  num_data = len(data)

  # removing remainder tokens
  remainder = num_data % math.floor(num_data/num_batches)
  print(remainder)
  data = data[:num_data - remainder]
  num_data = len(data)

  # batches = np.array_split(data, math.floor(num_data)/batch_size)
  batches = np.split(np.array(data),num_batches,axis=0)
  return batches

batches_words = gen_batches(5, 30, train)
batches_int = gen_batches(5, 30, int_train)
# batches_embeddings = gen_batches(5, 30, embeddings)

28
28


In [11]:
def print_seq(batch, window, seq_idx):
  input_tokens = batch[seq_idx:seq_idx+window]
  target_token = batch[seq_idx+window]

  print("input : ", input_tokens)
  print("target: [", target_token, "]\n")

In [12]:
print('--- batch 01 ---')
print_seq(batches_words[0], 5, 0)
print_seq(batches_words[0], 5, 1)
print_seq(batches_words[0], 5, 2)

print('-- batch 02 --')
print_seq(batches_words[1], 5, 0)
print_seq(batches_words[1], 5, 1)
print_seq(batches_words[1], 5, 2)

--- batch 01 ---
input :  ['<s>' 'the' 'battle' 'of' 'fort']
target: [ donelson ]

input :  ['the' 'battle' 'of' 'fort' 'donelson']
target: [ was ]

input :  ['battle' 'of' 'fort' 'donelson' 'was']
target: [ fought ]

-- batch 02 --
input :  ['the' 'battle' 'was' 'over' '</s>']
target: [ <s> ]

input :  ['battle' 'was' 'over' '</s>' '<s>']
target: [ 72 ]

input :  ['was' 'over' '</s>' '<s>' '72']
target: [ </s> ]



## Task 2: Bengio Style Feedforward network language model
- TensorFlow version: `2.1.0`

In [21]:
class BengioParams():

  def __init__(self, name, vocab_dict):
    self.name = name
    
    self.context_window = 5
    self.num_batches = 30

    self.vocab_len = len(vocab_dict)
    
    self.hidden_units = 50
    self.embeddings_dim = 60
    self.num_epochs = 20

    self.learning_rate = 0.5

    self.gpu_mem = 0.25
    
    self.tf_precision = tf.float32
    self.np_precision = np.float32

    self.init_scale = 0.5
    self.max_grad = 10.0


In [14]:
class BengioModel():
  """
  Class implements Bengio NN model with Tensorflow accoring to the function:
    y = b + Wx + Utanh(d + Hx)
  
  and 
    cost = softmax_cross_entropy?

  """

  def __init__(self, params):

    self.Y = tf.placeholder(
        dtype=params.tf_precision,
        shape=(params.num_batches, params.vocab_len),
        name="Y"
    )
    
    self.X = tf.placeholder(
        tf.int32, 
        shape=(params.num_batches, params.context_window),
        name="X"
    )

    # embeddings
    self.C = tf.Variable(
        tf.truncated_normal(
            shape=(params.vocab_len, params.embeddings_dim),
            mean=-1,
            stddev=-1
        ),
        dtype=params.tf_precision,
        name="C"
    )

    self.W = tf.Variable(
        tf.random_normal(
            shape=(params.vocab_len, params.context_window * params.embeddings_dim)
        ),
        name="W",
        dtype=params.tf_precision
    )
    
    self.H = tf.Variable(
        tf.random_normal(
            shape=(params.hidden_units, params.context_window * params.embeddings_dim)
        ),
        name="H",
        dtype=params.tf_precision
    )

    self.d = tf.Variable(
        tf.random_normal(
            shape=(params.hidden_units,)
        ),
        name="d",
        dtype=params.tf_precision
    )

    self.U = tf.Variable(
        tf.random_normal(
            (params.vocab_len, params.hidden_units)
        ),
        name="U",
        dtype=params.tf_precision
    )

    self.b = tf.Variable(
        tf.random_normal(
            shape=(params.vocab_len, )
        ),
        name="b",
        dtype=params.tf_precision
    )

    with tf.name_scope("Projection_Layer"):
      x = tf.nn.embedding_lookup(self.C, self.X)
      x = tf.reshape(
          x,
          shape=(params.num_batches, params.context_window * params.embeddings_dim)
      )

    with tf.name_scope("Hidden_Layer"):
      Hx = tf.matmul(x, tf.transpose(self.H))
      a = tf.nn.tanh(tf.add(Hx, self.d))

    with tf.name_scope("Output_Layer"):
      Ua = tf.matmul(a, tf.transpose(self.U))
      Wx = tf.matmul(x, tf.transpose(self.W))
      Y_hat = tf.add(self.b, tf.add(Wx, Ua)) 

    with tf.name_scope("Cost"):
     self.cost = tf.reduce_mean( 
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=self.Y,
            logits=Y_hat
        )
      )
     self.perplexity = tf.exp(self.cost)

    self.optimizer = tf.train.GradientDescentOptimizer(params.learning_rate).minimize(self.cost)

In [22]:
saver = tf.train.Saver()

def spiltInputTarget(batch, win_idx, params):
  _x = batch[win_idx:win_idx + params.context_window]
  _y = np.zeros(params.vocab_len)
  _y[batch[win_idx + params.context_window]] = 1
  return _x, _y


def run(model, params, batches_train_int, batches_validation_int, batches_test_int):

  perplexity_history = []
  cost_history = []

  val_perplexity_history = []
  val_cost_history = []

  test_perplexity_history = []
  test_cost_history = []
  

  gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.25)
  with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=True)) as sess:
    initializer = tf.global_variables_initializer()
    initializer.run()
    step = 0
    for epoch in range(params.num_epochs):
      # run model with 30 batches for a window size
      # for idx in total number of window sizes
        # for batch in batch_int
      win_idx = 0
      while win_idx < (len(batches_train_int[0]) - params.context_window - 1):
        batch_x = []
        batch_y = []
        for batch in batches_train_int:
          _x, _y = spiltInputTarget(batch, win_idx, params)
          batch_x.append(_x)
          batch_y.append(_y)
        
        cost, perplexity, _ = sess.run(
            [model.cost, model.perplexity, model.optimizer], 
            feed_dict={ model.X:batch_x, model.Y:batch_y }
        )

        # calculate validation & test preplexity after each epoch

        if step % 1000 == 0:
          print("train: step {}, cost: {}, perplexity: {}".format(step, cost, perplexity))
          perplexity_history.append(perplexity)
          cost_history.append(cost)
        
        step+=1
        win_idx+=1
        
      # save model after each epoch
      save_path = saver.save(sess, os.path.join(data_path, "/tmp/" + params.name +"_" + str(epoch+1) + ".ckpt"))
      print("Model saved in path: %s" % save_path)

      # validation
      win_idx = 0
      val_perplexity = 0
      val_cost = 0
      while win_idx < (len(batches_validation_int[0]) - params.context_window - 1):
        val_batch_x = []
        val_batch_y = []
        for batch in batches_validation_int:
          _x, _y = spiltInputTarget(batch, win_idx, params)
          val_batch_x.append(_x)
          val_batch_y.append(_y)

        cost, perplexity = sess.run(
            [model.cost, model.perplexity], 
            feed_dict={ model.X:val_batch_x, model.Y:val_batch_y }
        )
        val_perplexity = perplexity
        val_cost = cost
        
        win_idx+=1
      
      val_perplexity_history.append(val_perplexity)
      val_cost_history.append(val_cost)
      print("validation: epoch {}, cost: {}, perplexity: {}".format(epoch+1, val_cost, val_perplexity))

      # test
      win_idx = 0
      test_perplexity = 0
      test_cost = 0
      while win_idx < (len(batches_test_int[0]) - params.context_window - 1):
        test_batch_x = []
        test_batch_y = []
        for batch in batches_test_int:
          _x, _y = spiltInputTarget(batch, win_idx, params)
          test_batch_x.append(_x)
          test_batch_y.append(_y)

        cost, perplexity = sess.run(
            [model.cost, model.perplexity], 
            feed_dict={ model.X:test_batch_x, model.Y:test_batch_y }
        )
        test_perplexity = perplexity
        test_cost = cost
        
        win_idx+=1
      
      test_perplexity_history.append(test_perplexity)
      test_cost_history.append(test_cost)
      print("test: epoch {}, cost: {}, perplexity: {}".format(epoch+1, test_cost, test_perplexity_history))

  return perplexity_history, cost_history, val_perplexity_history, val_cost_history, test_perplexity_history, test_cost_history


In [0]:
# brown

# read the brown corpus text file
with open(os.path.join(data_path, 'brown_tokenized.txt'), 'r') as f:
  brown_tokens_all = f.readline().split(" ")

# remove tokens with less than 3 freq
brown_tok_freq = {}
for idx, tok in enumerate(brown_tokens_all):
  brown_tok_freq[tok] = brown_tok_freq.get(tok, 0) + 1

brown_tokens_all = [tok for tok in filter(lambda x: brown_tok_freq[x] >= 3, brown_tokens_all)]  

# create vocab
brown_vocab = set([tok for tok in brown_tokens_all])

# create train, validation, test
brown_train = brown_tokens_all[:800000]
brown_validation = brown_tokens_all[800000:1000000]
brown_test = brown_tokens_all[1000000:]

print("size of brown vocab: %i" % len(brown_vocab))

# integer representation
brown_vocab_dict = {}
for i, v in enumerate(brown_vocab):
    brown_vocab_dict[v] = i

brown_train_int = [brown_vocab_dict[tok] for tok in brown_train]
brown_validation_int = [brown_vocab_dict[tok] for tok in brown_validation]
brown_test_int = [brown_vocab_dict[tok] for tok in brown_test]

In [0]:
# train on brown
# brown_train_int

brown_params = BengioParams(brown_vocab_dict)
brown_model = BengioModel(name="p2_brown", params=brown_params)

brown_train_batches_int = gen_batches(5, 30, brown_train_int)
brown_val_batches_int = gen_batches(5, 30, brown_validation_int)
brown_test_batches_int = gen_batches(5, 30, brown_test_int)

brown_perplexity_history,  brown_cost_history, brown_val_perplexity_history, brown_val_cost_history, brown_test_perplexity_history, brown_test_cost_history = run(brown_model, brown_params, brown_train_batches_int, brown_val_batches_int, brown_test_batches_int)

In [0]:
# save brown files
import pickle

pickle.dump(brown_perplexity_history, open(os.path.join(data_path, 'brown_perplexity_history.p'), 'wb'))
pickle.dump(brown_cost_history, open(os.path.join(data_path, 'brown_cost_history.p'), 'wb'))
pickle.dump(brown_val_perplexity_history, open(os.path.join(data_path, 'brown_val_perplexity_history.p'), 'wb'))
pickle.dump(brown_val_cost_history, open(os.path.join(data_path, 'brown_val_cost_history.p'), 'wb'))
pickle.dump(brown_test_perplexity_history, open(os.path.join(data_path, 'brown_test_perplexity_history.p'), 'wb'))
pickle.dump(brown_test_cost_history, open(os.path.join(data_path, 'brown_test_cost_history.p'), 'wb'))

In [23]:
# train our model
class BengioModel2():
  """
  Class implements Bengio NN model with Tensorflow accoring to the function:
    y = b + Utanh(d + Hx)
  
  and 
    cost = softmax_cross_entropy?

  """

  def __init__(self, params):

    self.Y = tf.placeholder(
        dtype=params.tf_precision,
        shape=(params.num_batches, params.vocab_len),
        name="Y"
    )
    
    self.X = tf.placeholder(
        tf.int32, 
        shape=(params.num_batches, params.context_window),
        name="X"
    )

    # embeddings
    self.C = tf.Variable(
        tf.truncated_normal(
            shape=(params.vocab_len, params.embeddings_dim),
            mean=-1,
            stddev=-1
        ),
        dtype=params.tf_precision,
        name="C"
    )

    # self.W = tf.Variable(
    #     tf.random_normal(
    #         shape=(params.vocab_len, params.context_window * params.embeddings_dim)
    #     ),
    #     name="W",
    #     dtype=params.tf_precision
    # )
    
    self.H = tf.Variable(
        tf.random_normal(
            shape=(params.hidden_units, params.context_window * params.embeddings_dim)
        ),
        name="H",
        dtype=params.tf_precision
    )

    self.d = tf.Variable(
        tf.random_normal(
            shape=(params.hidden_units,)
        ),
        name="d",
        dtype=params.tf_precision
    )

    self.U = tf.Variable(
        tf.random_normal(
            (params.vocab_len, params.hidden_units)
        ),
        name="U",
        dtype=params.tf_precision
    )

    self.b = tf.Variable(
        tf.random_normal(
            shape=(params.vocab_len, )
        ),
        name="b",
        dtype=params.tf_precision
    )

    with tf.name_scope("Projection_Layer"):
      x = tf.nn.embedding_lookup(self.C, self.X)
      x = tf.reshape(
          x,
          shape=(params.num_batches, params.context_window * params.embeddings_dim)
      )

    with tf.name_scope("Hidden_Layer"):
      Hx = tf.matmul(x, tf.transpose(self.H))
      a = tf.nn.tanh(tf.add(Hx, self.d))

    with tf.name_scope("Output_Layer"):
      Ua = tf.matmul(a, tf.transpose(self.U))
      # Wx = tf.matmul(x, tf.transpose(self.W))
      Y_hat = tf.add(self.b, Ua) 

    with tf.name_scope("Cost"):
     self.cost = tf.reduce_mean( 
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=self.Y,
            logits=Y_hat
        )
      )
     self.perplexity = tf.exp(self.cost)

    self.optimizer = tf.train.GradientDescentOptimizer(params.learning_rate).minimize(self.cost)

int_train = pickle.load(open(os.path.join(data_path, 'group2.int_train.p'), 'rb'))
int_validation = pickle.load(open(os.path.join(data_path, 'group2.int_valid.p'), 'rb'))
int_test = pickle.load(open(os.path.join(data_path, 'group2.int_test.p'), 'rb'))
vocab_dict = pickle.load(open(os.path.join(data_path, 'group2.vocab_dict.p'), 'rb'))

train_batches_int = gen_batches(5, 30, int_train)
val_batches_int = gen_batches(5, 30, int_validation)
test_batches_int = gen_batches(5, 30, int_test)

run01_params = BengioParams(name="p2_run02", vocab_dict=vocab_dict)
run01_params.embedding_dimensions = 100
run01_params.hidden_units = 100
run01_params.context_window = 5
run01_params.gpu_mem = .9

run01_model = BengioModel2(params=run01_params)

run01_perplexity_history,  run01_cost_history, run01_val_perplexity_history, run01_val_cost_history, run01_test_perplexity_history, run01_test_cost_history = run(run01_model, run01_params, train_batches_int, val_batches_int, test_batches_int)

28
6
7
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device

train: step 0, cost: 45.23798751831055, perplexity: 4.432087830904596e+19
train: step 1000, cost: 9.845108985900879, perplexity: 18865.85546875
train: step 2000, cost: 10.878351211547852, perplexity: 53016.1171875
train: step 3000, cost: 9.460535049438477, perplexity: 12842.75390625
train: step 4000, cost: 9.95299243927002, perplexity: 21015.013671875
train: step 5000, cost: 9.27669906616211, perplexity: 10686.099609375
train: step 6000, cost: 8.430233001708984, perplexity: 4583.56787109375
train: step 7000, cost: 7.677794456481934, perplexity: 2159.850830078125
train: step 8000, cost: 8.279935836791992, perplexity: 3943.941162109375
train: step 9000, cost: 11.067889213562012, perplex

train: step 110000, cost: 7.141575336456299, perplexity: 1263.4171142578125
train: step 111000, cost: 6.967850208282471, perplexity: 1061.937255859375
train: step 112000, cost: 9.32162094116211, perplexity: 11177.0849609375
train: step 113000, cost: 8.383399963378906, perplexity: 4373.8544921875
train: step 114000, cost: 7.622907638549805, perplexity: 2044.498046875
train: step 115000, cost: 10.156434059143066, perplexity: 25756.287109375
train: step 116000, cost: 7.6013336181640625, perplexity: 2000.8624267578125
train: step 117000, cost: 6.535459518432617, perplexity: 689.1504516601562
train: step 118000, cost: 8.75282096862793, perplexity: 6328.51513671875
train: step 119000, cost: 12.518902778625488, perplexity: 273457.84375
train: step 120000, cost: 7.076120853424072, perplexity: 1183.369140625
train: step 121000, cost: 11.365167617797852, perplexity: 86264.0
train: step 122000, cost: 8.725149154663086, perplexity: 6155.79443359375
train: step 123000, cost: 7.898001194000244, perp

train: step 219000, cost: 7.138317584991455, perplexity: 1259.307861328125
train: step 220000, cost: 8.987711906433105, perplexity: 8004.12158203125
train: step 221000, cost: 8.285526275634766, perplexity: 3966.051513671875
train: step 222000, cost: 8.017208099365234, perplexity: 3032.698486328125
train: step 223000, cost: 9.032891273498535, perplexity: 8374.0361328125
train: step 224000, cost: 7.706980228424072, perplexity: 2223.816650390625
train: step 225000, cost: 11.35206127166748, perplexity: 85140.765625
train: step 226000, cost: 7.750917434692383, perplexity: 2323.703125
train: step 227000, cost: 8.051082611083984, perplexity: 3137.189453125
train: step 228000, cost: 11.471220016479492, perplexity: 95915.2265625
train: step 229000, cost: 7.715759754180908, perplexity: 2243.4267578125
train: step 230000, cost: 8.104205131530762, perplexity: 3308.35107421875
train: step 231000, cost: 9.074076652526855, perplexity: 8726.125
train: step 232000, cost: 7.2422332763671875, perplexity:

train: step 329000, cost: 12.746240615844727, perplexity: 343259.03125
train: step 330000, cost: 8.88845157623291, perplexity: 7247.78759765625
train: step 331000, cost: 7.84929895401001, perplexity: 2563.936279296875
train: step 332000, cost: 8.297170639038086, perplexity: 4012.50341796875
train: step 333000, cost: 8.605420112609863, perplexity: 5461.1796875
train: step 334000, cost: 8.115525245666504, perplexity: 3346.0146484375
train: step 335000, cost: 9.31087589263916, perplexity: 11057.62890625
train: step 336000, cost: 8.915857315063477, perplexity: 7449.16552734375
train: step 337000, cost: 8.984391212463379, perplexity: 7977.58642578125
train: step 338000, cost: 8.277371406555176, perplexity: 3933.84033203125
train: step 339000, cost: 7.778325080871582, perplexity: 2388.271240234375
train: step 340000, cost: 11.212956428527832, perplexity: 74084.109375
train: step 341000, cost: 8.277788162231445, perplexity: 3935.47998046875
train: step 342000, cost: 8.572175979614258, perplex

train: step 441000, cost: 9.83181381225586, perplexity: 18616.69140625
train: step 442000, cost: 9.399762153625488, perplexity: 12085.505859375
train: step 443000, cost: 9.636190414428711, perplexity: 15308.9111328125
train: step 444000, cost: 8.501935958862305, perplexity: 4924.29296875
train: step 445000, cost: 8.079543113708496, perplexity: 3227.75830078125
train: step 446000, cost: 9.492588996887207, perplexity: 13261.083984375
train: step 447000, cost: 8.033513069152832, perplexity: 3082.552001953125
train: step 448000, cost: 6.814467430114746, perplexity: 910.9312744140625
train: step 449000, cost: 8.943676948547363, perplexity: 7659.30810546875
train: step 450000, cost: 7.854365825653076, perplexity: 2576.960205078125
train: step 451000, cost: 11.022356033325195, perplexity: 61227.76171875
train: step 452000, cost: 8.510138511657715, perplexity: 4964.8505859375
train: step 453000, cost: 8.265935897827148, perplexity: 3889.11083984375
train: step 454000, cost: 10.500885009765625,

...

train: step 2617000, cost: 7.769913673400879, perplexity: 2368.266845703125
train: step 2618000, cost: 9.075132369995117, perplexity: 8735.341796875
train: step 2619000, cost: 12.180275917053223, perplexity: 194906.625
train: step 2620000, cost: 8.492648124694824, perplexity: 4878.7685546875
train: step 2621000, cost: 13.250543594360352, perplexity: 568378.9375
train: step 2622000, cost: 8.938643455505371, perplexity: 7620.8515625
Model saved in path: /tmp/p2_run02_17.ckpt
validation: epoch 17, cost: 8.481911659240723, perplexity: 4826.66796875
test: epoch 17, cost: 8.686893463134766, perplexity: [19173.516, 17592.662, 17449.03, 11225.486, 21339.318, 24617.826, 22408.908, 66997.37, 14599.063, 22131.621, 45416.027, 64620.516, 16979.926, 27348.324, 9064.601, 10224.9375, 5924.748]
train: step 2623000, cost: 7.589444637298584, perplexity: 1977.215087890625
train: step 2624000, cost: 7.176100730895996, perplexity: 1307.798828125
train: step 2625000, cost: 8.97564697265625, perplexity: 7908.

train: step 2724000, cost: 8.249105453491211, perplexity: 3824.203125
train: step 2725000, cost: 8.903288841247559, perplexity: 7356.126953125
train: step 2726000, cost: 7.610358238220215, perplexity: 2019.001220703125
train: step 2727000, cost: 7.345581531524658, perplexity: 1549.335693359375
train: step 2728000, cost: 8.038165092468262, perplexity: 3096.925537109375
train: step 2729000, cost: 7.7507476806640625, perplexity: 2323.308837890625
train: step 2730000, cost: 8.422337532043457, perplexity: 4547.52099609375
train: step 2731000, cost: 9.05941390991211, perplexity: 8599.109375
train: step 2732000, cost: 8.39813232421875, perplexity: 4438.76904296875
train: step 2733000, cost: 7.231152534484863, perplexity: 1381.814208984375
train: step 2734000, cost: 6.095449924468994, perplexity: 443.8337097167969
train: step 2735000, cost: 8.740519523620605, perplexity: 6251.142578125
train: step 2736000, cost: 6.475783348083496, perplexity: 649.2276000976562
train: step 2737000, cost: 10.189

train: step 2830000, cost: 7.727166652679443, perplexity: 2269.16357421875
train: step 2831000, cost: 7.215689182281494, perplexity: 1360.611083984375
train: step 2832000, cost: 8.08340835571289, perplexity: 3240.258544921875
train: step 2833000, cost: 9.800416946411133, perplexity: 18041.265625
train: step 2834000, cost: 8.41364860534668, perplexity: 4508.1787109375
train: step 2835000, cost: 8.979598999023438, perplexity: 7939.44775390625
train: step 2836000, cost: 8.88962173461914, perplexity: 7256.27392578125
train: step 2837000, cost: 7.490929126739502, perplexity: 1791.716064453125
train: step 2838000, cost: 7.443770408630371, perplexity: 1709.182373046875
train: step 2839000, cost: 8.157320976257324, perplexity: 3488.8271484375
train: step 2840000, cost: 9.468786239624023, perplexity: 12949.1611328125
train: step 2841000, cost: 14.149883270263672, perplexity: 1397063.75
train: step 2842000, cost: 11.451373100280762, perplexity: 94030.3671875
train: step 2843000, cost: 7.47066640

train: step 2936000, cost: 8.866436004638672, perplexity: 7089.96728515625
train: step 2937000, cost: 8.4152250289917, perplexity: 4515.29150390625
train: step 2938000, cost: 7.755971431732178, perplexity: 2335.476806640625
train: step 2939000, cost: 9.866782188415527, perplexity: 19279.203125
train: step 2940000, cost: 8.46503734588623, perplexity: 4745.90478515625
train: step 2941000, cost: 10.166437149047852, perplexity: 26015.224609375
train: step 2942000, cost: 10.450687408447266, perplexity: 34568.12890625
train: step 2943000, cost: 8.279313087463379, perplexity: 3941.48583984375
train: step 2944000, cost: 9.811365127563477, perplexity: 18239.87109375
train: step 2945000, cost: 7.729816913604736, perplexity: 2275.185546875
train: step 2946000, cost: 7.79551887512207, perplexity: 2429.68994140625
train: step 2947000, cost: 9.298680305480957, perplexity: 10923.59375
train: step 2948000, cost: 10.87765121459961, perplexity: 52979.01953125
train: step 2949000, cost: 8.102804183959961

train: step 3047000, cost: 8.414143562316895, perplexity: 4510.41064453125
train: step 3048000, cost: 7.418241024017334, perplexity: 1666.100341796875
train: step 3049000, cost: 8.998927116394043, perplexity: 8094.39501953125
train: step 3050000, cost: 9.956154823303223, perplexity: 21081.578125
train: step 3051000, cost: 7.528724193572998, perplexity: 1860.72998046875
train: step 3052000, cost: 8.035162925720215, perplexity: 3087.641845703125
train: step 3053000, cost: 8.703815460205078, perplexity: 6025.8603515625
train: step 3054000, cost: 7.663182735443115, perplexity: 2128.521240234375
train: step 3055000, cost: 9.299700736999512, perplexity: 10934.74609375
train: step 3056000, cost: 8.837141990661621, perplexity: 6885.2861328125
train: step 3057000, cost: 8.000601768493652, perplexity: 2982.75244140625
train: step 3058000, cost: 11.35775375366211, perplexity: 85626.8125
train: step 3059000, cost: 11.904065132141113, perplexity: 147866.5
train: step 3060000, cost: 8.25148391723632

In [24]:
pickle.dump(run01_perplexity_history, open(os.path.join(data_path, 'p2_run01_perplexity_history.p'), 'wb'))
pickle.dump(run01_cost_history, open(os.path.join(data_path, 'p2_run01_cost_history.p'), 'wb'))
pickle.dump(run01_val_perplexity_history, open(os.path.join(data_path, 'p2_run01_val_perplexity_history.p'), 'wb'))
pickle.dump(run01_val_cost_history, open(os.path.join(data_path, 'p2_run01_val_cost_history.p'), 'wb'))
pickle.dump(run01_test_perplexity_history, open(os.path.join(data_path, 'p2_run01_test_perplexity_history.p'), 'wb'))
pickle.dump(run01_test_cost_history, open(os.path.join(data_path, 'p2_run01_test_cost_history.p'), 'wb'))